In [1]:
# 题目一、信用评分。
# 为了降低不良贷款率，保障自身资金安全，提高风险控制水平，银行等金融机构会根据客户的信用历史资料构建信用评分模型给客户评分。根据客户的信用得分，可以估计客户按时还款的可能，并据此决定是否发放贷款及贷款的额度和利率。
# 1、获取数据：信用评分卡模型.xlsx(2分)
import pandas as pd
import  numpy as np
a1=pd.read_excel('E:\\python代码\\lx-yk\\data-yk\\yk6\信用评分卡模型.xlsx')
print(a1)
# 2、数据探索分析：
# 1）查看前5行（2分）
print('查看前5行\n', a1.head(5))
# 2）查看数据信息（2分）
print("查看数据信息\n",a1.info)
# 3）查看基本统计信息（2分）
print('查看基本统计信息\n',a1.describe())
# 4）查看数据的行数和列数（2分）
print('查看数据的行数和列数\n',a1.shape)
# 3、数据清洗：
# 1）缺失值处理：查看数据缺失情况，采用合适的方式填充或者删除（5分）
# 查看数据缺失情况
a1.isnull().any()
#如果有空  填充或者删除
a1.fillna(a1.mean(),inplace=True)#
# 再次查看数据缺失情况
a1.isnull().any()
# 2）重复值处理：检查是否有重复值，有则删除（5分
# 检查是否有重复值
a1.drop_duplicates(inplace=True)
# 4、自主选择合适的方法进行特征选择(3分)
x=a1.drop('信用评分',axis=1)
y=a1['信用评分']
y
# 5、划分测试集与训练集（2分）
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
# 将特征缩放一下
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

       月收入  年龄  性别  历史授信额度  历史违约次数  信用评分
0     7783  29   0   32274       3    73
1     7836  40   1    6681       4    72
2     6398  25   0   26038       2    74
3     6483  23   1   24584       4    65
4     5167  23   1    6710       3    73
..     ...  ..  ..     ...     ...   ...
995  12873  52   1   58190       2    87
996  11478  36   0   95688       1    81
997  14105  45   1   76221       2    89
998  11914  37   1   39906       2    83
999  14467  45   0   84052       2    83

[1000 rows x 6 columns]
查看前5行
     月收入  年龄  性别  历史授信额度  历史违约次数  信用评分
0  7783  29   0   32274       3    73
1  7836  40   1    6681       4    72
2  6398  25   0   26038       2    74
3  6483  23   1   24584       4    65
4  5167  23   1    6710       3    73
查看数据信息
 <bound method DataFrame.info of        月收入  年龄  性别  历史授信额度  历史违约次数  信用评分
0     7783  29   0   32274       3    73
1     7836  40   1    6681       4    72
2     6398  25   0   26038       2    74
3     6483  23   1   24584       4    65
4  

In [2]:
# 6、模型选择与训练：XGBoost、GBDT、逻辑回归、随机森林或者其他你任务合适的模型。（至少2种）（8分）
# XGBoost
from xgboost import XGBRegressor
bst = XGBRegressor()
bst.fit(x_train, y_train)
bst_yc=bst.predict(x_test)

# 随机森林
from sklearn.ensemble import RandomForestRegressor
sj = RandomForestRegressor()
sj.fit(x_train, y_train)
sj_yc=sj.predict(x_test)



In [10]:
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error
# 7、使用网格搜索进行参数调优（5分）
from sklearn.model_selection import GridSearchCV
bst_cs={'learning_rate':[0.1,0.2,0.3],'max_depth':[2,3,4]}

bst_cv = GridSearchCV(bst, bst_cs,cv=5)
bst_cv.fit(x_train, y_train)
bst_t7=bst_cv.best_params_
print("XGBoost调优后的参数",bst_t7)
sj_cs={'n_estimators':[100,200,300],'max_depth':[1,10,20]}
sj_cv = GridSearchCV(sj, sj_cs,cv=5)
sj_cv.fit(x_train, y_train)
sj_t7=sj_cv.best_params_
print("随机森林调优后的参数",sj_t7)


# 8、模型评估：使用R2 、MSE、RMSE或者你认为合适的指标评估模型。（至少两种）（5分）
#XGBoost
bst_gjq=bst_cv.best_estimator_
ty_XGB=bst_gjq.predict(x_test)
from sklearn.metrics import r2_score
R2=r2_score(y_test, ty_XGB)
MSE=mean_squared_error(y_test, ty_XGB)
RMSE=np.sqrt(MSE)#计算平方根函数
print(f"使用R2 {R2}MSE{MSE}RMSE:计算平方根函数{RMSE}")

#随机森林
sj_gjq=sj_cv.best_estimator_
ty_sj=sj_gjq.predict(x_test)
R21=r2_score(y_test, ty_sj)
MSE1=mean_squared_error(y_test, ty_sj)
RMSE1=np.sqrt(MSE)#计算平方根函数
print(f"使用R2 {R21}MSE{MSE1}RMSE:计算平方根函数{RMSE1}")



# 9、查看特征重要性（3分）
t9=bst_gjq.feature_importances_

t91=sj_gjq.feature_importances_
print(t91,t9)



# 10、模型保存：使用合适的方式保存训练好的模型。（3分）

joblib.dump(bst_gjq, 'a1.pkl')  
joblib.dump(sj_gjq, 'a2.pkl')  



XGBoost调优后的参数 {'learning_rate': 0.1, 'max_depth': 3}
随机森林调优后的参数 {'max_depth': 10, 'n_estimators': 200}
使用R2 0.6911657552857946MSE20.287344222823968RMSE:计算平方根函数4.504147446834303
使用R2 0.6909562564673385MSE20.301106215598537RMSE:计算平方根函数4.504147446834303
[0.22524451 0.14861367 0.00714734 0.5057501  0.11324438] [0.05125443 0.16087697 0.01481416 0.27191195 0.5011425 ]


['a2.pkl']